# IMPORTS

In [1]:
from chord_extractor.extractors import Chordino
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mingus.core import chords
from midiutil import MIDIFile

# VARIABLES

In [15]:
conversion_file_path = '../audio_output/gravity__architects__anavae/accompaniment.mp3'
NOTES = ['C', 'C#', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'Ab', 'A', 'Bb', 'B']
OCTAVES = list(range(len(NOTES)-1))
NOTES_IN_OCTAVE = len(NOTES)
OCTAVE = 4 

# FUNCTIONS

In [20]:
def get_chord_composition(chord_shorthand):
    try:
        chord_composition = chords.from_shorthand(chord_shorthand)
        return chord_composition
    except: 
        return None

def swap_accidentals(note):
    note_swap_dict = {
        'Db': 'C#',
        'D#': 'Eb',
        'Fb': 'E',
        'E#': 'F',
        'Gb': 'F#',
        'G#': 'Ab',
        'A#': 'Bb',
        'B#': 'C',
        'Cb': 'B',
        'Ebb': 'D'
    }
    if note in note_swap_dict.keys():
        note = note_swap_dict[note]
    return note

def apply_function_to_note_list(note_list, function):
    if note_list:
        return list(map(function, note_list))

def note_to_number(note):
    note = NOTES.index(note)
    note += NOTES_IN_OCTAVE * OCTAVE
    return note

# Inputs

In [21]:
chordino = Chordino(roll_on=0)
chords_extraction = chordino.extract(conversion_file_path, sr=22050*4)

KeyboardInterrupt: 

# DATA TRANSFORMATIONS

In [22]:
chord_list = [extracted_chord.chord for extracted_chord in chords_extraction]
t_list = [extracted_chord.timestamp/60 for extracted_chord in chords_extraction]

In [23]:
chord_information = pd.DataFrame()
chord_information['chord'] = chord_list
chord_information['timestamp'] = t_list
chord_information['chord_composition'] = chord_information.chord.apply(get_chord_composition)
chord_information['chord_composition_fixed'] = chord_information.chord_composition.apply(apply_function_to_note_list, function=swap_accidentals)
chord_information['chord_note_nb'] = chord_information.chord_composition_fixed.apply(apply_function_to_note_list, function=note_to_number)
chord_information['chord_duration'] = chord_information.timestamp.fillna(0).rolling(window=2).apply(lambda x: x.iloc[1] - x.iloc[0])
chord_information['chord_duration'] = list(chord_information['chord_duration'])[1:] + [np.nan]
chord_information = chord_information.dropna()

In [24]:
#for i, pitch in enumerate(array_of_note_numbers):
#    MyMIDI.addNote(track, channel, pitch, time + i, duration, volume)
track = 0
channel = 0
time = 0  # In beats
tempo = 60  # In BPM
volume = 100  # 0-127, as per the MIDI standard

MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
MyMIDI.addTempo(track, time, tempo)

for row_index in range(chord_information.shape[0]):
    row = chord_information.iloc[row_index]
    time = row.timestamp*60
    duration = row.chord_duration*60-0.1
    for musical_note in (row.chord_note_nb):
        pitch = musical_note
        MyMIDI.addNote(track, channel, pitch, time, duration, volume)

# OUTPUT

In [26]:
with open("song_midi_test3.mid", "wb") as output_file:
    MyMIDI.writeFile(output_file)